In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D
from tensorflow.keras.models import Model

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0
print(x_train.shape)

In [ ]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print(x_train.shape)

In [ ]:
K = len(set(y_train))
print(K)

In [ ]:
#Building a model, using functional API
def create_model():
    i = Input(shape=x_train[0].shape)
    x = Conv2D(32, (3,3), strides=2, activation='relu')(i)
    x = Conv2D(64, (3,3), strides=2, activation='relu')(x)
    x = Conv2D(128, (3,3), strides=2, activation='relu')(x)
    x = GlobalMaxPooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(K, activation='softmax')(x)

    model = Model(i,x)
    
    return model

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    model = create_model()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
r = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size = 128, epochs=15)

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
plt.plot(r.history['accuracy'], label='accuracy')
plt.plot(r.history['val_accuracy'], label='val_accuracy')
plt.legend()

# Plot Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion Matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
        print("Normalized Confusion Matrix")
    else:
        print("Confusion Matrix without Normalization")
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment='center',
                 color='white' if cm[i,j] > thresh else 'black')
    plt.tight_layout()
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()
    
p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [ ]:
labels = '''
0
1
2
3
4
5
6
7
8
9
'''.split()

In [ ]:
misclassified_idx = np.where(p_test != y_test)[0]
i = np.random.choice(misclassified_idx)
plt.imshow(x_test[i].reshape(28,28), cmap='gray')
plt.title("True Label: %s Predicted Label: %s" % (labels[y_test[i]], labels[p_test[i]]))
print(np.where(p_test != y_test))

In [ ]:
image = Image.open('42.jpg')

In [ ]:
greyscaled = image.resize((28,28)).convert('L')

In [ ]:
inverted = ImageOps.invert(greyscaled)

In [ ]:
img_as_arr = np.asarray(inverted)

In [ ]:
img_as_arr2 = img_as_arr.reshape(28,28,1)

In [ ]:
img_as_arr2.shape

In [ ]:
a = model.predict(img_as_arr2)

In [ ]:
a

In [ ]:
plt.imshow(img_as_arr2.reshape(28,28), cmap='gray')
print("Predicted Label: ", labels[int(np.where(a == np.amax(a))[1])])

In [ ]:
model.save('NumClassifier.pb')